In [69]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window
import talib

In [70]:
global DoEval #是否進行模型評估

#DoEval = False
# next_date_range = ["20180604", "20180605", "20180606", "20180607", "20180608"] #設定預測區間
# next_date_range = ["20180528", "20180529", "20180530", "20180531", "20180601"] #設定預測區間
# next_date_range = ["20180521", "20180522", "20180523", "20180524", "20180525"] #設定預測區間
DoEval = True
# next_date_range = ["20180514", "20180515", "20180516", "20180517", "20180518"] #設定預測區間
ignore_dates = [""]#設定排除日(如端午節)
next_date_range = ["20180507", "20180508", "20180509", "20180510", "20180511"] #設定預測區間
predict_start_date = next_date_range[0]

if sc.master[0:5]=="local":
#     Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180601"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180511"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"
else:
    Path = "hdfs://master:9000/user/hduser"
Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180608"

In [71]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [72]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [73]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19575
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [74]:
#取出etf的distinct id
etf_ids = []
etf_idx_dic = {}
etf_idx = 0
for row in tetfp_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
    etf_idx_dic.update({row["etf_id"]: etf_idx})
    etf_idx += 1
etf_idx_dic
# etf_ids = ['0050   ']

{'0050   ': 0,
 '0051   ': 1,
 '0052   ': 2,
 '0053   ': 3,
 '0054   ': 4,
 '0055   ': 5,
 '0056   ': 6,
 '0057   ': 7,
 '0058   ': 8,
 '0059   ': 9,
 '006201 ': 10,
 '006203 ': 11,
 '006204 ': 12,
 '006208 ': 13,
 '00690  ': 14,
 '00692  ': 15,
 '00701  ': 16,
 '00713  ': 17}

In [75]:
etf_dic = {}
for etfid in etf_ids:
    export_dt = tetfp_dt.filter("etf_id='" +etfid+ "' and etf_date < '" + predict_start_date + "'") \
        .orderBy("etf_id", "etf_date", ascending=True)
    export_pd = export_dt.toPandas()
    etf_dic.update({etfid.strip(): export_pd})

In [76]:
#functions of TA lib
#print(talib.get_functions())

In [77]:
etf_dic["0050"]["etf_close"].count()

1307

In [78]:
import math
def get_feature_pre(curr_list):
    #在feature前加上一個Nan後回傳, 第二個值則回傳原本的最後一個值作為下次的feature
    if type(curr_list) is pd.Series:
        rtn_list = curr_list.tolist()
    else:
        rtn_list = curr_list
    rtn_list = [None] + rtn_list
    return (rtn_list[:-1], [rtn_list[-1]])

def checkNan(num):
    if num == None:
        return True
    elif math.isnan(num):
        return True
    else:
        return False
    
def calculate_n_bias(close_price, ma):
#     (close price - MA)/MA
    if checkNan(close_price) or checkNan(ma):
        return None
    else:
        return (close_price - ma)/ma
    
def calculate_n_bias_list(close_ser, ma_list):
    #calculate n bias (20 days)
    bias_list = []
    for cprice, ma in zip(close_ser.tolist(), ma_list):
        bias_list.append(calculate_n_bias(cprice, ma))
    return bias_list

def calculate_bias_ma_list(ma3_list, ma6_list):
    #calculate bias_ma = (ma3-ma6)/ma6
    bias_list = []
    for ma3, ma6 in zip(ma3_list, ma6_list):
        bias_list.append(calculate_n_bias(ma3, ma6))
    return bias_list

def calculate_bias_3_6_list(bias3_list, bias6_list):
    #bias_3_6 = bias3 - bias6
    bias_list = []
    for bias3, bias6 in zip(bias3_list, bias6_list):
        if checkNan(bias3) or checkNan(bias6):
            bias_list.append(None)
        else:
            bias_list.append(bias3 - bias6)
    return bias_list

def calculate_dif(ema12_list, ema26_list):
    #差離值DIF = 12日EMA - 26日EMA 
    dif_list = []
    for ema12, ema26 in zip(ema12_list, ema26_list):
        if checkNan(ema12) or checkNan(ema26):
            dif_list.append(None)
        else:
            dif_list.append(ema12 - ema26)
    return dif_list
def calculate_slope(cur_list, prev_list):
    slope_list = []
    for cur, prev in zip(cur_list, prev_list):
        if checkNan(cur) or checkNan(prev):
            slope_list.append(None)
        else:
            slope_list.append((cur-prev)/prev)
    return slope_list

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import itertools
def create_feature(etf_pd, next_date, scaler_dic):
    close_ser = etf_pd["etf_close"]
    #EMA
    ema3 = talib.EMA(close_ser,timeperiod=3)
    ema3_prev, _ = get_feature_pre(ema3)
    ema3_slope = calculate_slope(ema3, ema3_prev)
    ema5 = talib.EMA(close_ser,timeperiod=5)
    ema5_prev, _ = get_feature_pre(ema5)
    ema5_slope = calculate_slope(ema5, ema5_prev)
    ema10 = talib.EMA(close_ser,timeperiod=10)
    ema10_prev, _ = get_feature_pre(ema10)
    ema10_slope = calculate_slope(ema10, ema10_prev)
    ema20 = talib.EMA(close_ser,timeperiod=20)
    ema30 = talib.EMA(close_ser,timeperiod=30)
    
    #make diff
    close_ser_prev, _ = get_feature_pre(close_ser)
    close_diff = calculate_dif(close_ser.tolist(), close_ser_prev)
    
    close_diff_2, _ = get_feature_pre(close_diff)
    close_diff_3, _ = get_feature_pre(close_diff_2)
    #BIAS
    #nBIAS -3, 6, 20 => (close price - MA)/MA   ,Paper 建議用20日MA
    ma3 = talib.MA(np.array(close_ser), timeperiod=3)
    ma6 = talib.MA(np.array(close_ser), timeperiod=6)
    ma20 = talib.MA(np.array(close_ser), timeperiod=20)
    nbias3 = calculate_n_bias_list(close_ser, ma3)
    nbias6 = calculate_n_bias_list(close_ser, ma6)
    nbias20 = calculate_n_bias_list(close_ser, ma20)
    
    #BIAS_ma = (ma3-ma6)/ma6
    bias_ma = calculate_bias_ma_list(ma3, ma6)
    
    #bias_3_6 = bias3 - bias6
    bias_3_6 = calculate_bias_3_6_list(nbias3, nbias6)
    
    #KD --> only use STOCHRSI
    k, d = talib.STOCHRSI(close_ser, timeperiod=9, fastk_period=3, fastd_period=3, fastd_matype=0)
    
    #差離值DIF = 12日EMA - 26日EMA 
    ema12 = talib.EMA(close_ser,timeperiod=12)
    ema26 = talib.EMA(close_ser,timeperiod=26)
    dif = calculate_dif(ema12, ema26)
    
    #MACD
#     macd1, macdsignal1, macdhist1 = talib.MACD(close_ser, fastperiod=12, slowperiod=26, signalperiod=9)
    macd1, macdsignal1, macdhist1 = talib.MACDEXT(close_ser, fastperiod=12, fastmatype=0, 
                                               slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#     macd1, macdsignal1, macdhist1 = talib.MACDFIX(close_ser, signalperiod=9)
    #RSI
    rsi = talib.RSI(close_ser, timeperiod=10)
    
    #CMO
    cmo = talib.CMO(close_ser, timeperiod=14)
    
    #ROCR - Rate of change ratio: (price/prevPrice)
    #rocr = talib.ROCR(close_ser, timeperiod=10)
    #ROC - Rate of change : ((price/prevPrice)-1)*100
    rocr = talib.ROC(close_ser, timeperiod=10)
    
    #PPO - Percentage Price Oscillator
    ppo = talib.PPO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    #APO - Absolute Price Oscillator
    apo = talib.APO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    
    #MOM - Momentum
    mom = talib.MOM(close_ser, timeperiod=10)
    #TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    trix = talib.TRIX(close_ser, timeperiod=30)
    #作標準化
    etfid = etf_pd["etf_id"].tolist()[0]
    tmp_pd = pd.DataFrame({"etf_id": etfid, "etf_date": etf_pd["etf_date"], "etf_close": close_ser, "close_dif": close_diff,  
#                            "close_diff_1": close_diff, "close_diff_2": close_diff_2, "close_diff_3": close_diff_3,
                           "ema3":ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                           "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                          "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                          "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    tmp_pd = tmp_pd[tmp_pd['etf_date'] > '20140101']
    #exclude outlier
#     for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         tmp_pd = tmp_pd[np.abs(tmp_pd[col] - tmp_pd[col].mean()) <= 3.0*tmp_pd[col].std()] #只取3倍以內標準差資料
    
    std_dic = {}
    for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         scaler_key = etfid.strip() + '-' + col
        scaler_key = col
        if scaler_key in scaler_dic:
            scaler = scaler_dic[scaler_key]
            merged = list(itertools.chain.from_iterable(scaler.transform(tmp_pd[[col]])))                
#             merged = tmp_pd[col]
        else:
            scaler = MinMaxScaler()
            merged = list(itertools.chain.from_iterable(scaler.fit_transform(tmp_pd[[col]])))
            merged = tmp_pd[col]
            #保留etf_close的scaler在後續使用
            scaler_dic.update({scaler_key: scaler})
        std_dic.update({col: merged})
    
    #作出目前的feature table及預測用的feature table
    ema3, next_ema3 = get_feature_pre(std_dic["ema3"])
    ema3_slope, next_ema3_slope = get_feature_pre(std_dic["ema3_slope"])
    ema5, next_ema5 = get_feature_pre(std_dic["ema5"])
    ema5_slope, next_ema5_slope = get_feature_pre(std_dic["ema5_slope"])
    ema10, next_ema10 = get_feature_pre(std_dic["ema10"])
    ema10_slope, next_ema10_slope = get_feature_pre(std_dic["ema10_slope"])
    ema20, next_ema20 = get_feature_pre(std_dic["ema20"])
    ema30, next_ema30 = get_feature_pre(std_dic["ema30"])
    
    nbias3, next_nbias3 = get_feature_pre(std_dic["nbias3"])
    nbias6, next_nbias6 = get_feature_pre(std_dic["nbias6"])
    nbias20, next_nbias20 = get_feature_pre(std_dic["nbias20"])
    
    bias_ma, next_bias_ma = get_feature_pre(std_dic["bias_ma"])
    bias_3_6, next_bias_3_6 = get_feature_pre(std_dic["bias_3_6"])
    k, next_k = get_feature_pre(std_dic['k'])
    d, next_d = get_feature_pre(std_dic['d'])
    dif, next_dif = get_feature_pre(std_dic['dif'])
    
    #macd, next_macd = get_feature_pre(std_dic['macd'])
    #macdsignal, next_macdsignal = get_feature_pre(std_dic['macdsignal'])
    #macdhist, next_macdhist = get_feature_pre(std_dic['macdhist'])
    macd1, next_macd1 = get_feature_pre(std_dic['macd1'])
    macdsignal1, next_macdsignal1 = get_feature_pre(std_dic['macdsignal1'])
    macdhist1, next_macdhist1 = get_feature_pre(std_dic['macdhist1'])
                                               
    rsi, next_rsi = get_feature_pre(std_dic['rsi'])
    cmo, next_cmo = get_feature_pre(std_dic['cmo'])
    rocr, next_rocr = get_feature_pre(std_dic['rocr'])
    ppo, next_ppo = get_feature_pre(std_dic['ppo'])
    
#     close_diff_1, next_close_diff_1 = get_feature_pre(std_dic['close_diff_1'])
#     close_diff_2, next_close_diff_2 = get_feature_pre(std_dic['close_diff_2'])
#     close_diff_3, next_close_diff_3 = get_feature_pre(std_dic['close_diff_3'])
    
    #"apo": apo, "mom": mom, "trix": trix
#     apo, next_apo = get_feature_pre(std_dic['apo'])
#     mom, next_mom = get_feature_pre(std_dic['mom'])
#     trix, next_trix = get_feature_pre(std_dic['trix'])
#     print(len(std_dic['etf_close']), ' ', len(close_diff_1), ' ', len(close_diff_2), ' ' , len(rsi))
    
    rtn_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": tmp_pd["etf_date"], 
                           "etf_close": std_dic["etf_close"], "close_dif": std_dic['close_dif'], 
                           'ema3': ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                          "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                           "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                           "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    #col = 'close_dif'
    #rtn_pd = rtn_pd[np.abs(rtn_pd[col] - rtn_pd[col].mean()) <= 3.0*rtn_pd[col].std()] #只取3倍以內標準差資料
    
    next_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": next_date, 
                            'etf_close': -1.0, "close_dif": -1.0, 
                           "ema3": next_ema3, "ema5": next_ema5, "ema10": next_ema10, "ema20": next_ema20, "ema30" : next_ema30,
                          "ema3_slope": next_ema3_slope, "ema5_slope": next_ema5_slope, "ema10_slope": next_ema10_slope,
                            "nbias3": next_nbias3, "nbias6": next_nbias6, "nbias20": next_nbias20, "bias_ma": next_bias_ma,
                          "bias_3_6": next_bias_3_6, "k": next_k, "d": next_d, "dif": next_dif,
                          #"macd": next_macd, "macdsignal": next_macdsignal, "macdhist": next_macdhist,
                            "macd1": next_macd1, "macdsignal1": next_macdsignal1, "macdhist1": next_macdhist1,
                          "rsi": next_rsi, "cmo": next_cmo, "rocr": next_rocr, "ppo": next_ppo})
    
    return (rtn_pd, next_pd, scaler_dic)

In [79]:
val = etf_dic["0050"][["etf_close"]]
# print(val["etf_close"].tolist())
from sklearn.preprocessing import StandardScaler
import itertools
scaler = StandardScaler()
scaler.fit_transform(val)
#merged = list(itertools.chain.from_iterable(scaler.fit_transform(val)))
#merged
scaler.inverse_transform([-1.44880072])[0]

54.34565034921221

In [80]:
#create feature pandas
etf_pd_dic = {}
next_pd_dic = {}
scaler_dic = {}
for etfid in etf_ids:
    etf_pd, next_pd, scalers = create_feature(etf_dic[etfid.strip()], predict_start_date, {})
    etf_pd_dic.update({etfid.strip() : etf_pd})
    next_pd_dic.update({etfid.strip() : next_pd})
    scaler_dic.update(scalers)

In [81]:
etf_pd_dic["0050"]['etf_close'].describe()

count    1060.000000
mean       69.833349
std         7.883596
min        55.800000
25%        63.737500
50%        68.825000
75%        74.062500
max        88.300000
Name: etf_close, dtype: float64

In [82]:
#etf_pd_dic["0054"].describe()

In [83]:
#etf_pd_dic["00692"].describe()

In [84]:
from sklearn.preprocessing import MinMaxScaler
import itertools
# pd_0050=etf_pd_dic["0051"]
# pd_0050=pd_0050[pd_0050.etf_date > '20180101']
# scaler = MinMaxScaler()
# pd_0050['close_dif']=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[['close_dif']])))
# # pd_0050[['close_dif']].plot()
# for col in list(pd_0050.columns.values): #["bias_3_6", "ema5_slope", 'ema10_slope']:
#     pd_0050[col]=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[[col]])))
#     pd_0050[['close_dif',col]].plot()
# plt.show()

In [85]:
non_feature_list = ['etf_id', 'etf_date', 'close_dif', 'etf_close', 'idx']
feature_cols = [col for col in list(etf_pd_dic["0050"].columns.values) if col not in non_feature_list]
# feature_cols = ['ema5']
print(feature_cols)

['bias_3_6', 'bias_ma', 'cmo', 'd', 'dif', 'ema10', 'ema10_slope', 'ema20', 'ema3', 'ema30', 'ema3_slope', 'ema5', 'ema5_slope', 'k', 'macd1', 'macdhist1', 'macdsignal1', 'nbias20', 'nbias3', 'nbias6', 'ppo', 'rocr', 'rsi']


In [86]:
train_dt_org = pd.concat([etf_pd_dic[etfid.strip()] for etfid in etf_ids])
std_dic = {}
for col in [c for c in train_dt_org.columns.values if c not in ["etf_id", "etf_date"]]:
        scaler_key = col
        if scaler_key in scaler_dic:
            scaler = scaler_dic[scaler_key]
            merged = list(itertools.chain.from_iterable(scaler.transform(train_dt_org[[col]])))                
        else:
            scaler = MinMaxScaler()
            merged = list(itertools.chain.from_iterable(scaler.fit_transform(train_dt_org[[col]])))
            #保留etf_close的scaler在後續使用
            scaler_dic.update({scaler_key: scaler})
        std_dic.update({col: merged})
train_dt = pd.DataFrame(std_dic)
print('train_dt count:', train_dt.shape)

for etfid in etf_ids:
    std_dic = {}
    next_pd = next_pd_dic[etfid.strip()]
    for col in [c for c in next_pd.columns.values if c not in ["etf_id", "etf_date"]]:
        scaler = scaler_dic[col]
        merged = list(itertools.chain.from_iterable(scaler.transform(next_pd[[col]])))
        std_dic.update({col: merged})
    next_pd_dic.update({etfid.strip() : pd.DataFrame(std_dic)})
print('next_pd_dic count:', len(next_pd_dic.keys()), '*' , next_pd_dic['0050'].shape)

train_dt count: (15531, 26)
next_pd_dic count: 18 * (1, 26)


In [87]:
import xgboost as xgb
def trainModel(train_dt, feature_cols):
    train_x1 = train_dt[feature_cols].values
    #model for close_dif
    train_y1 = train_dt["close_dif"].values
    #$$$ parameter
    model1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
        colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
    model1.fit(train_x1, train_y1)
    #model for etf_close
    train_y2 = train_dt["etf_close"].values
    model2 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
        colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
    model2.fit(train_x1, train_y2)
    return (model1, model2)

In [88]:
pd_0050=etf_dic["0050"]
prev_date = pd_0050.loc[~pd_0050["etf_date"].isin(next_date_range)]["etf_date"].max()
prev_date

'20180504'

In [89]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#計算上或下的值(udf)
def judge_up_down_pred_native(curr_price, prev_close_price):
    if checkNan(prev_close_price):
        return 0.0
    else:
        prev_price = prev_close_price
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred_native, DoubleType())

In [90]:
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
#eval dt for evluate score
tetf_dt_eval = tetfp_dt.filter(tetfp_dt.etf_date.isin([prev_date] + next_date_range)) \
    .withColumn("prev_close", lag(tetfp_dt.etf_close).over(wsSpec_etf)) 
tetf_dt_eval =  tetf_dt_eval.withColumn("act_ud", judge_up_down_pred(tetf_dt_eval.etf_close, tetf_dt_eval.prev_close))
tetf_dt_eval = tetf_dt_eval.filter(tetf_dt_eval.etf_date.isin(next_date_range))
tetf_dt_eval.cache()
tetf_dt_eval.show()

+-------+--------+----------------+--------+--------+-------+---------+---------+----------+------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|prev_close|act_ud|
+-------+--------+----------------+--------+--------+-------+---------+---------+----------+------+
|00701  |20180507|   國泰臺灣低波動30    |    20.9|    20.9|  20.71|    20.75|     36.0|     20.75|   0.0|
|00701  |20180508|   國泰臺灣低波動30    |   20.74|   20.85|  20.74|    20.82|     43.0|     20.75|   1.0|
|00701  |20180509|   國泰臺灣低波動30    |   20.82|    20.9|  20.82|    20.89|     49.0|     20.82|   1.0|
|00701  |20180510|   國泰臺灣低波動30    |   20.89|   20.92|  20.89|    20.89|     13.0|     20.89|   0.0|
|00701  |20180511|   國泰臺灣低波動30    |   21.02|   21.11|  21.02|    21.11|     73.0|     20.89|   1.0|
|0051   |20180507|元大中型100         |   31.96|   32.03|  31.88|     32.0|     16.0|      31.8|   1.0|
|0051   |20180508|元大中型100         |    32.0|   32.24|   32.0|    32.24|      6.0|      32.0|   1.0|


In [91]:
tetf_pd_eval = tetf_dt_eval.toPandas()
# tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == '006203 ') & (tetf_pd_eval['etf_date'] == '20180522')]['etf_close'].values[0]

In [92]:
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
def doEvaluate(predict_res_final, tetf_pd_eval):
    etf_close_list = []
    act_ud_list = []
    prediction_list = []
    pred_ud_list = []
    acc_list = []
    weights = [0.1, 0.15, 0.2, 0.25, 0.3]
    judge_score = 0.0
    for row in predict_res_final:
        eid = row['etf_id']
        edate = row['etf_date']
        if edate not in next_date_range:
            continue
        etf_eval = tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == eid) & (tetf_pd_eval['etf_date'] == edate)]
        etf_close = etf_eval['etf_close'].values[0]
        act_ud = etf_eval['act_ud'].values[0]
        prediction = row['prediction']
        pred_ud = row['pred_ud']
        #etf score
        eidx = next_date_range.index(edate)
        judge_score = judge_score + \
            ((0.5 if pred_ud == act_ud else 0.0) + \
             ((etf_close-abs(prediction-etf_close))/etf_close)*0.5)*weights[eidx]
        etf_close_list.append(etf_close)
        act_ud_list.append(act_ud)
        prediction_list.append(prediction)
        pred_ud_list.append(pred_ud)
        acc_list.append(1.0 if pred_ud == act_ud else 0.0)
    #evaluate accuracy_score
    accuracy = accuracy_score(act_ud_list, pred_ud_list, normalize=True)
#     print(act_ud_list)
#     print(pred_ud_list)
    rmse = mean_squared_error(etf_close_list, prediction_list)
    return (rmse, accuracy, judge_score)
    

In [93]:
scaler_dic.keys()

dict_keys(['bias_3_6', 'bias_ma', 'close_dif', 'cmo', 'd', 'dif', 'ema10', 'ema10_slope', 'ema20', 'ema3', 'ema30', 'ema3_slope', 'ema5', 'ema5_slope', 'etf_close', 'k', 'macd1', 'macdhist1', 'macdsignal1', 'nbias20', 'nbias3', 'nbias6', 'ppo', 'rocr', 'rsi', 'idx'])

In [94]:
def doPredict1(next_date_range, model1, etf_dic, next_pd_dic, scaler_dic, feature_cols):
    predict_res_final_1 = []
    dummy_date = '20999999'
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model1.predict(data=pred_data)
#             print(pred_res)

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
#             print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['close_dif']
            prediction = prev_close + scaler.inverse_transform(pred_res[0])[0][0]
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_1.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if (date != next_date_range[-1]) and (date != dummy_date):
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_1
    
def doPredict2(next_date_range, model2, etf_dic, next_pd_dic, scaler_dic, feature_cols):
    predict_res_final_2 = []
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model2.predict(data=pred_data)
#             print(eid.strip(), '-', date, ': ', pred_res[0], '-', 
#                   scaler_dic[eid.strip()+'-etf_close'].inverse_transform(pred_res[0])[0][0])

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
    #         print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['etf_close']
            prediction = scaler.inverse_transform(pred_res[0])[0][0]
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_2.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if date != next_date_range[-1]:
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_2

In [95]:
# for subset in itertools.combinations(feature_cols, 1):
#     print(list(subset))

In [ ]:
def is_feature_prefixed(feature_str, feature_prefix):
    if feature_prefix == None or len(feature_prefix) == 0:
        return True
    for feature in feature_prefix:
        if feature_str.startswith(feature):
            return True
    return False

In [ ]:
type_list = []
feature_list = []
acc_list = []
rmse_list = []
escore_list = []
# feature_prefix = []
feature_prefix = [
    'ema20,ema10_slope',
    'ema10,ema5_slope',
    'ema10_slope,ema20',
    'ema30,ema10_slope',
    'ema10,nbias6',
    'nbias6,ema10',
    'ema5_slope,ema10',
    'ema30,rocr',
    'ema20,ema5_slope',
    'rocr,ema30',
    'nbias6,ema20',
    'ema3_slope,ema10',
    'nbias3,ema5',
    'ema5_slope,ema20',
    'ema30,ppo'

] # for find third feature
# feature_prefix = [
#     'bias_ma,ema20,macdsignal1','bias_ma,ema20,rsi',
#     'bias_3_6,ema20,macdsignal1','bias_ma,ema30,macdhist1',
#     'bias_3_6,ema20,macdhist1','ema5,nbias3,rocr',
#     'bias_ma,ema30,macdsignal1','bias_3_6,ema20,ppo',
#     'bias_3_6,ema20,ema30','bias_3_6,ema20,macd1',
#     'bias_ma,ema20,ppo','bias_ma,ema20,k',
#     'bias_ma,ema30,macd1','bias_3_6,ema20,nbias6',
#     'bias_ma,ema30,macdsignal1','bias_3_6,ema30,macdsignal1'
# ]
feature_num = 3
thresh_hold = 14.9
over_thresh_hold_num = 0
max_sample_num = 10
for features in itertools.permutations(feature_cols, feature_num):
    feature_str = ','.join(list(features))
    if not is_feature_prefixed(feature_str, feature_prefix):
        continue
    if feature_num == 2 and over_thresh_hold_num > max_sample_num:
        continue
    model1, model2 = trainModel(train_dt, list(features))
    #recover to re-run
    for eid in etf_ids:
        etfid = eid
        etf_dt = etf_dic[eid.strip()]
        etf_dt = etf_dt.loc[~etf_dt["etf_date"].isin(next_date_range)]
        etf_dic.update({eid.strip(): etf_dt})
        etf_pd, next_pd, _ = create_feature(etf_dic[etfid.strip()], predict_start_date, scaler_dic)
        etf_pd_dic.update({etfid.strip() : etf_pd})
        next_pd_dic.update({etfid.strip() : next_pd})
    #evaluate close_dif
    predict_res_pd_1 = doPredict1(next_date_range, model1, etf_dic, next_pd_dic, scaler_dic, list(features))
    rmse1, acc1, e_score1 = doEvaluate(predict_res_pd_1, tetf_pd_eval)
    if e_score1 > thresh_hold:
        over_thresh_hold_num += 1
    type_list.append("1")
    feature_list.append(feature_str)
    acc_list.append(acc1)
    rmse_list.append(rmse1)
    escore_list.append(e_score1)
    #recover to re-run
    for eid in etf_ids:
        etfid = eid
        etf_dt = etf_dic[eid.strip()]
        etf_dt = etf_dt.loc[~etf_dt["etf_date"].isin(next_date_range)]
        etf_dic.update({eid.strip(): etf_dt})
        etf_pd, next_pd, _ = create_feature(etf_dic[etfid.strip()], predict_start_date, scaler_dic)
        etf_pd_dic.update({etfid.strip() : etf_pd})
        next_pd_dic.update({etfid.strip() : next_pd})
    #evaluate etf_close
    predict_res_pd_2 = doPredict2(next_date_range, model2, etf_dic, next_pd_dic, scaler_dic, list(features))
    rmse2, acc2, e_score2 = doEvaluate(predict_res_pd_2, tetf_pd_eval)
    if e_score2 > thresh_hold:
        over_thresh_hold_num += 1
    type_list.append("2")
    feature_list.append(feature_str)
    acc_list.append(acc2)
    rmse_list.append(rmse2)
    escore_list.append(e_score2)
    print(feature_str, ': final1=', e_score1, ', final2=', e_score2)

score_dic = {
    'feature_type' : type_list, 'feature': feature_list, 'accuracy': acc_list, 'rmse': rmse_list,
    'final_score': escore_list
}


ema10,ema5_slope,bias_3_6 : final1= 13.128883884145768 , final2= 14.818764826555512
ema10,ema5_slope,bias_ma : final1= 12.716155639941762 , final2= 15.162670066250625
ema10,ema5_slope,cmo : final1= 13.118116212559361 , final2= 13.43512878601517
ema10,ema5_slope,d : final1= 13.117837093620327 , final2= 15.68834443023279
ema10,ema5_slope,dif : final1= 13.234883486789142 , final2= 13.543669953184283
ema10,ema5_slope,ema10_slope : final1= 14.340443964268315 , final2= 15.035143926837595
ema10,ema5_slope,ema20 : final1= 13.928222337895933 , final2= 14.160542055267234
ema10,ema5_slope,ema3 : final1= 13.607939555638042 , final2= 13.631634651662758
ema10,ema5_slope,ema30 : final1= 13.15407618577699 , final2= 14.318663731788101
ema10,ema5_slope,ema3_slope : final1= 13.296756498885722 , final2= 15.63958734614353
ema10,ema5_slope,ema5 : final1= 13.593993506902304 , final2= 13.579670325612728
ema10,ema5_slope,k : final1= 13.523339486767421 , final2= 14.45037888306428
ema10,ema5_slope,macd1 : final1

ema20,ema5_slope,ppo : final1= 12.862051373410566 , final2= 14.067032132787322
ema20,ema5_slope,rocr : final1= 14.26016588833289 , final2= 15.176386131984042
ema20,ema5_slope,rsi : final1= 13.455333117987673 , final2= 13.301435945162247
ema30,ema10_slope,bias_3_6 : final1= 13.099305641400072 , final2= 15.117555346761856
ema30,ema10_slope,bias_ma : final1= 13.476536691670894 , final2= 15.298560222881454
ema30,ema10_slope,cmo : final1= 13.507208008954548 , final2= 13.23831444481981
ema30,ema10_slope,d : final1= 12.516607526957133 , final2= 15.244442328514639
ema30,ema10_slope,dif : final1= 13.41927966367957 , final2= 13.162270322913596
ema30,ema10_slope,ema10 : final1= 13.041497246283026 , final2= 13.203298153771208
ema30,ema10_slope,ema20 : final1= 13.50327137704721 , final2= 15.370567778693998
ema30,ema10_slope,ema3 : final1= 13.663980157515878 , final2= 13.688274768712404
ema30,ema10_slope,ema3_slope : final1= 13.752458331844252 , final2= 15.770030731579903
ema30,ema10_slope,ema5 : fi

ema3_slope,ema10_slope,nbias3 : final1= 13.147474473267536 , final2= 5.812726628348555
ema3_slope,ema10_slope,nbias6 : final1= 13.708632384762451 , final2= 3.7863415236724256
ema3_slope,ema10_slope,ppo : final1= 12.533953485283671 , final2= 9.117444369690896
ema3_slope,ema10_slope,rocr : final1= 13.907873772174078 , final2= 4.656363032635803
ema3_slope,ema10_slope,rsi : final1= 13.10747934592287 , final2= 9.099172541950624
ema5_slope,ema10,bias_3_6 : final1= 13.21701295750136 , final2= 15.285840547036866
ema5_slope,ema10,bias_ma : final1= 13.748013709770829 , final2= 14.97289618731825
ema5_slope,ema10,cmo : final1= 13.544641598386653 , final2= 14.231321273609087
ema5_slope,ema10,d : final1= 13.77110104693951 , final2= 15.156516221761533
ema5_slope,ema10,dif : final1= 13.437037669947863 , final2= 14.337655526122754
ema5_slope,ema10,ema10_slope : final1= 14.262006854294201 , final2= 14.052301154867349
ema5_slope,ema10,ema20 : final1= 14.38123311419429 , final2= 14.852407741501144
ema5_sl

In [ ]:
# etf_dic['0050']
# score_dic['feature']

In [ ]:
score_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'accuracy', 'rmse', 'final_score']]

In [ ]:
score_pd.sort_values(by=['final_score'], ascending=[0]).head(15)

In [ ]:
score_pd.sort_values(by=['accuracy'], ascending=[0]).head()

In [ ]:
score_pd.sort_values(by=['rmse'], ascending=[1]).head()